In [1]:
pip install snowflake-connector-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import getpass
pwd_approved=getpass.getpass("Enter password:")

Enter password:········


In [4]:
pip install pyarrow

In [7]:
import snowflake.connector
conn_approved = snowflake.connector.connect(user='sowbhagya22',password=str(pwd_approved),account='mx53414.ap-south-1.aws')

In [8]:
sql_approved='select * from LOANS.S_LOANS.APPROVED_LOAN'

In [9]:
res_approved=conn.cursor().execute(sql_approved).fetchall()

KeyboardInterrupt: 

In [9]:
df=pd.read_sql(sql_approved,conn_approved)
df.info()

C:\Users\sowbhagya.vodal\AppData\Local\Temp\ipykernel_3712\4222303541.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(sql_approved,conn_approved)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551452 entries, 0 to 551451
Columns: 148 entries, id to Label
dtypes: float64(102), int64(1), object(45)
memory usage: 622.7+ MB


In [133]:
pd.isnull(df).any()

id                        True
member_id                 True
loan_amnt                 True
funded_amnt               True
funded_amnt_inv           True
                         ...  
settlement_percentage     True
settlement_term           True
Quarter                  False
Year                     False
Label                    False
Length: 148, dtype: bool

In [134]:
df['int_rate'] = df['int_rate'].str.rstrip('%').astype(float)

In [135]:
print(df)

                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [136]:
def risk_score(mths_since_last_record):
    if pd.isnull(mths_since_last_record):  # Check for missing values
        return 80
    elif mths_since_last_record >= 24:
        return 60
    elif mths_since_last_record >= 12:
        return 30
    elif mths_since_last_record >= 6:
        return 20
    else:
        return 10

In [137]:
df['Risk_Score'] = df['mths_since_last_record'].apply(risk_score)







In [138]:
print(df)

                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [139]:
print(df['earliest_cr_line'])

0         Jan-2010
1         Dec-2010
2         Mar-1993
3         Mar-1997
4         Oct-2007
            ...   
551447    Feb-1994
551448    May-1998
551449    Jul-2007
551450        None
551451        None
Name: earliest_cr_line, Length: 551452, dtype: object


In [140]:
from datetime import datetime


In [141]:
import math

In [142]:
unique_values = df['earliest_cr_line'].unique()

In [143]:
print(unique_values)

['Jan-2010' 'Dec-2010' 'Mar-1993' 'Mar-1997' 'Oct-2007' 'Nov-1996'
 'Jan-2006' 'Mar-2006' 'Oct-2001' 'Aug-2005' 'Jan-1984' 'Jan-1995'
 'Apr-2006' 'Sep-2005' 'May-2013' 'May-2006' 'Mar-1995' 'Feb-1999'
 'Jul-2003' 'Sep-2013' 'Aug-1994' 'Aug-2002' 'Aug-2001' 'Aug-2003'
 'Sep-2006' 'Apr-2010' 'Dec-1994' 'Apr-2002' 'Jan-2011' 'Apr-1999'
 'Sep-2008' 'Jul-1998' 'Jan-2005' 'Jul-2008' 'May-1997' 'Jul-1996'
 'Aug-1977' 'Oct-1997' 'Sep-1999' 'Aug-1978' 'Jul-1994' 'Jun-1993'
 'May-1999' 'Nov-2012' 'Mar-2005' 'Sep-2000' 'Dec-2013' 'Aug-2006'
 'Apr-2000' 'Mar-1984' 'Aug-2004' 'Nov-1999' 'Apr-2008' 'May-2000'
 'Sep-2001' 'Mar-2007' 'Feb-2004' 'Jun-1999' 'Aug-1998' 'Nov-2004'
 'Feb-2003' 'Jan-1999' 'Jun-1998' 'Aug-2000' 'Sep-2002' 'Feb-1998'
 'Jul-1983' 'Oct-2008' 'Jun-2004' 'Feb-1989' 'May-1991' 'Dec-2005'
 'Jan-2004' 'Jul-2014' 'Aug-2011' 'Apr-2011' 'Aug-2012' 'Dec-2003'
 'Feb-1996' 'Jul-2013' 'Feb-1997' 'Oct-1983' 'Jul-1988' 'Apr-2004'
 'Apr-1991' 'Oct-2006' 'Feb-1988' 'Nov-2002' 'Jun-1996' 'Apr-2

In [111]:
def calculate_credit_history(date_str):
    if date_str is None:  
        return None  # Handle missing values
    
    earliest_cr_line = datetime.strptime(date_str, '%b-%Y')
    date_present = datetime.now()
    date_diff_days = (earliest_cr_line - date_present).days
    
    # Calculate the result as per the formula
    result = date_diff_days // (1000 * 60 * 60 * 24 * 30)
    
    return result







In [144]:
def calculate_credit_history(date_str):
    if date_str is None:  
        return None  # Handle missing values
    
    earliest_cr_line = datetime.strptime(date_str, '%b-%Y')
    date_present = datetime.now()
    date_diff = (date_present.year - earliest_cr_line.year) * 12 + (date_present.month - earliest_cr_line.month)

    # Convert the result to an integer by flooring it
    date_diff = int(date_diff)
    
    return date_diff


In [145]:
print(calculate_credit_history)

<function calculate_credit_history at 0x000001ED1E603920>


In [146]:
print(df)

                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [147]:
df['Credit_History'] = df['earliest_cr_line'].apply(calculate_credit_history)

In [148]:
print(df['Credit_History'])

0         163.0
1         152.0
2         365.0
3         317.0
4         190.0
          ...  
551447    354.0
551448    303.0
551449    193.0
551450      NaN
551451      NaN
Name: Credit_History, Length: 551452, dtype: float64


In [149]:
min_value = df['Credit_History'].min()
max_value = df['Credit_History'].max()

# Print the results
print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: 102.0
Maximum value: 1074.0


In [150]:
def update_risk_score(row):
    if row['Credit_History'] >= 48:
        return row['Risk_Score'] + 80
    elif row['Credit_History'] >= 24:
        return row['Risk_Score'] + 60
    elif row['Credit_History'] >= 12:
        return row['Risk_Score'] + 40
    else:
        return row['Risk_Score'] + 10

In [151]:
df['Risk_Score'] = df.apply(update_risk_score, axis=1)

In [152]:
print(df['Risk_Score'])

0         160
1         160
2         160
3         160
4         160
         ... 
551447    160
551448    160
551449    160
551450     90
551451     90
Name: Risk_Score, Length: 551452, dtype: int64


In [153]:
def update_risk_score1(row):
    inq_last_6mths = row['inq_last_6mths']
    risk_score = row['Risk_Score']
    
    if inq_last_6mths >= 4:
        return risk_score + 20
    elif inq_last_6mths >= 3:
        return risk_score + 30
    elif inq_last_6mths >= 2:
        return risk_score + 50
    elif inq_last_6mths >= 1:
        return risk_score + 60
    else:
        return risk_score + 80

In [154]:
df['Risk_Score'] = df.apply(update_risk_score1, axis=1)

In [155]:
print(df['Risk_Score'])

0         240
1         240
2         240
3         240
4         220
         ... 
551447    240
551448    240
551449    220
551450    170
551451    170
Name: Risk_Score, Length: 551452, dtype: int64


In [156]:
def update_risk_score2(row):
    num_actv_bc_tl = row['num_actv_bc_tl']
    risk_score = row['Risk_Score']
    
    if num_actv_bc_tl >= 4:
        return risk_score + 80
    elif num_actv_bc_tl >= 3:
        return risk_score + 60
    elif num_actv_bc_tl >= 2:
        return risk_score + 50
    elif num_actv_bc_tl >= 1:
        return risk_score + 30
    else:
        return risk_score + 10

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score2, axis=1)

print(df)

                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [157]:
def update_risk_score3(row):
    dti = row['dti']
    risk_score = row['Risk_Score']
    
    if dti >= 500:
        return risk_score + 10
    elif dti >= 200:
        return risk_score + 20
    elif dti >= 100:
        return risk_score + 30
    elif dti >= 50:
        return risk_score + 50
    elif dti >= 20:
        return risk_score + 60
    elif dti >= 10:
        return risk_score + 70
    else:
        return risk_score + 80

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score3, axis=1)

print(df)







                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [158]:
def update_risk_score4(row):
    mort_acc = row['mort_acc']
    risk_score = row['Risk_Score']
    
    if mort_acc >= 10:
        return risk_score + 10
    elif mort_acc >= 5:
        return risk_score + 30
    elif mort_acc >= 4:
        return risk_score + 40
    elif mort_acc >= 3:
        return risk_score + 50
    elif mort_acc >= 2:
        return risk_score + 60
    elif mort_acc >= 1:
        return risk_score + 70
    else:
        return risk_score + 80

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score4, axis=1)

print(df)


                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [159]:
def update_risk_score5(row):
    hardship_flag = row['hardship_flag']
    risk_score = row['Risk_Score']
    
    if hardship_flag == 'Y':
        return risk_score + 0
    else:
        return risk_score + 40

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score5, axis=1)

print(df)

                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [160]:
def update_risk_score6(row):
    debt_settlement_flag = row['debt_settlement_flag']
    risk_score = row['Risk_Score']
    
    if debt_settlement_flag == 'Y':
        return risk_score + 0
    else:
        return risk_score + 40

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score6, axis=1)

print(df)






                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [161]:
def update_risk_score7(row):
    loan_status = row['loan_status']
    risk_score = row['Risk_Score']
    
    if loan_status == 'Default':
        return risk_score + 0
    elif loan_status == 'Charged Off':
        return risk_score + 10
    elif loan_status == 'Late (31-120 days)':
        return risk_score + 20
    elif loan_status == 'Late (16-30 days)':
        return risk_score + 30
    elif loan_status == 'In Grace Period':
        return risk_score + 40
    elif loan_status == 'Fully Paid':
        return risk_score + 50
    else:
        return risk_score + 80

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score7, axis=1)

print(df)





                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [162]:
def update_risk_score8(row):
    int_rate = row['int_rate']
    risk_score = row['Risk_Score']
    
    if int_rate >= 20:
        return risk_score + 80
    elif int_rate >= 15:
        return risk_score + 60
    elif int_rate >= 10:
        return risk_score + 30
    elif int_rate >= 5:
        return risk_score + 10
    else:
        return risk_score + 0

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score8, axis=1)

print(df)


                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [163]:
def update_risk_score9(row):
    pub_rec_bankruptcies = row['pub_rec_bankruptcies']
    risk_score = row['Risk_Score']
    
    if pub_rec_bankruptcies >= 5:
        return risk_score + 0
    elif pub_rec_bankruptcies >= 4:
        return risk_score + 10
    elif pub_rec_bankruptcies >= 3:
        return risk_score + 20
    elif pub_rec_bankruptcies >= 2:
        return risk_score + 30
    elif pub_rec_bankruptcies >= 1:
        return risk_score + 40
    else:
        return risk_score + 80

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score9, axis=1)

print(df)


                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [164]:
def update_risk_score10(row):
    revol_bal = row['revol_bal']
    risk_score = row['Risk_Score']
    
    if pd.isna(revol_bal):  # MISSING_NUMERIC condition
        return risk_score + 30
    elif revol_bal >= 100000:
        return risk_score + 20
    elif revol_bal >= 75000:
        return risk_score + 30
    elif revol_bal >= 50000:
        return risk_score + 40
    elif revol_bal >= 10000:
        return risk_score + 50
    elif revol_bal >= 100:
        return risk_score + 80
    else:
        return risk_score + 60

# Apply the function to update the 'Risk_Score' column
df['Risk_Score'] = df.apply(update_risk_score10, axis=1)

print(df)






                                                      id member_id  loan_amnt  \
0                                                   None      None    20000.0   
1                                                   None      None     7000.0   
2                                                   None      None     6700.0   
3                                                   None      None    18000.0   
4                                                   None      None     1000.0   
...                                                  ...       ...        ...   
551447                                              None      None     4375.0   
551448                                              None      None    12000.0   
551449                                              None      None    14000.0   
551450  Total amount funded in policy code 1: 1741781700      None        NaN   
551451   Total amount funded in policy code 2: 539397275      None        NaN   

        funded_amnt  funded

In [165]:
column_names=df.columns
print(column_names)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term', 'Quarter', 'Year', 'Label',
       'Risk_Score', 'Credit_History'],
      dtype='object', length=150)


In [166]:
df.rename(columns={'Label': 'Application_status'}, inplace=True)

In [167]:
df.rename(columns={'title':'Loan Title'}, inplace=True)

In [168]:
df.rename(columns={'emp_length':'Employment Length'}, inplace=True)

In [169]:
df.rename(columns={'dti':'Debt-To-Income Ratio'}, inplace=True)

In [170]:
df.rename(columns={'loan_amnt':'Amount Requested'}, inplace=True)

In [171]:
print(df)

                                                      id member_id  \
0                                                   None      None   
1                                                   None      None   
2                                                   None      None   
3                                                   None      None   
4                                                   None      None   
...                                                  ...       ...   
551447                                              None      None   
551448                                              None      None   
551449                                              None      None   
551450  Total amount funded in policy code 1: 1741781700      None   
551451   Total amount funded in policy code 2: 539397275      None   

        Amount Requested  funded_amnt  funded_amnt_inv       term  int_rate  \
0                20000.0      20000.0          20000.0  36 months     10.42   
1

In [172]:
print(df['Amount Requested'])

0         20000.0
1          7000.0
2          6700.0
3         18000.0
4          1000.0
           ...   
551447     4375.0
551448    12000.0
551449    14000.0
551450        NaN
551451        NaN
Name: Amount Requested, Length: 551452, dtype: float64


In [173]:
df['Amount Requested'].isnull().any()

True

In [174]:
column_name='Amount Requested'
null_rows = df.loc[df[column_name].isnull()]
print(null_rows)





                                                      id member_id  \
115904  Total amount funded in policy code 1: 1817354125      None   
115905   Total amount funded in policy code 2: 592843163      None   
260618  Total amount funded in policy code 1: 1437969475      None   
260619   Total amount funded in policy code 2: 519691874      None   
366071  Total amount funded in policy code 1: 1538432075      None   
366072   Total amount funded in policy code 2: 603225638      None   
488774  Total amount funded in policy code 1: 1791201400      None   
488775   Total amount funded in policy code 2: 640990301      None   
551450  Total amount funded in policy code 1: 1741781700      None   
551451   Total amount funded in policy code 2: 539397275      None   

        Amount Requested  funded_amnt  funded_amnt_inv  term  int_rate  \
115904               NaN          NaN              NaN  None       NaN   
115905               NaN          NaN              NaN  None       NaN   
260618 

In [175]:
column_name = 'Amount Requested'

# Delete rows with null values in the specified column
df = df.dropna(subset=[column_name])

In [176]:
print(df.count())

id                         0
member_id                  0
Amount Requested      551442
funded_amnt           551442
funded_amnt_inv       551442
                       ...  
Quarter               551442
Year                  551442
Application_status    551442
Risk_Score            551442
Credit_History        551442
Length: 150, dtype: int64


In [177]:
df['Application_status'].isnull().any()

False

In [178]:
df['Debt-To-Income Ratio'].isnull().any()

True

In [179]:
column_name='Debt-To-Income Ratio'

null_rows = df.loc[df[column_name].isnull()]
print(null_rows)

          id member_id  Amount Requested  funded_amnt  funded_amnt_inv  \
421     None      None           40000.0      40000.0          40000.0   
1325    None      None           24000.0      24000.0          24000.0   
1511    None      None            4200.0       4200.0           4200.0   
2035    None      None           26000.0      26000.0          26000.0   
2250    None      None            7000.0       7000.0           7000.0   
...      ...       ...               ...          ...              ...   
547565  None      None            6000.0       6000.0           6000.0   
548650  None      None           10000.0      10000.0          10000.0   
548800  None      None           10000.0      10000.0          10000.0   
550823  None      None           28000.0      28000.0          28000.0   
551322  None      None           24000.0      24000.0          23975.0   

             term  int_rate  installment grade sub_grade  ...  \
421     36 months     14.08      1368.66     C

In [180]:
df['Employment Length'].isnull().any()

False

In [181]:
df['Loan Title'].isnull().any()

False

In [182]:
column_name = 'Employment Length' 
value_to_find = 'n/a'
# Print rows with the specified value in the specified column 
rows_with_value = df.loc[df[column_name] == value_to_find]

In [183]:
column_name1 = 'Loan Title' 
value_to_find1 = 'Other'
# Print rows with the specified value in the specified column 
rows_with_value1 = df.loc[df[column_name1] == value_to_find1]

In [184]:
print(rows_with_value1)

          id member_id  Amount Requested  funded_amnt  funded_amnt_inv  \
0       None      None           20000.0      20000.0          20000.0   
1       None      None            7000.0       7000.0           7000.0   
4       None      None            1000.0       1000.0           1000.0   
12      None      None            4800.0       4800.0           4800.0   
16      None      None           20000.0      20000.0          20000.0   
...      ...       ...               ...          ...              ...   
551371  None      None            8600.0       8600.0           8575.0   
551399  None      None           35000.0      35000.0          34975.0   
551407  None      None           15000.0      15000.0          14950.0   
551409  None      None           11000.0      11000.0          10975.0   
551423  None      None            1500.0       1500.0           1500.0   

             term  int_rate  installment grade sub_grade  ...  \
0       36 months     10.42       649.30     B

In [187]:
# Specify the columns you want to extract
wanted_columns = ['Amount Requested','Application_status','Debt-To-Income Ratio','Employment Length','Loan Title','Risk_Score']
condition_column = 'Employment Length'
condition_value ='n/a'
# Extract specific columns and rows with non-null values
df = df.loc[(df['Debt-To-Income Ratio'].notnull()) & (df[condition_column] != condition_value)&(df['Loan Title'] != 'Other'), wanted_columns]

print(df)







        Amount Requested Application_status  Debt-To-Income Ratio  \
2                 6700.0           Approved                 20.00   
3                18000.0           Approved                 25.09   
5                12000.0           Approved                  4.90   
6                10500.0           Approved                 35.57   
7                 8000.0           Approved                 18.39   
...                  ...                ...                   ...   
551445           18000.0           Approved                 15.37   
551446            6000.0           Approved                 28.93   
551447            4375.0           Approved                 33.72   
551448           12000.0           Approved                 11.10   
551449           14000.0           Approved                  1.35   

       Employment Length               Loan Title  Risk_Score  
2                8 years       Debt consolidation         780  
3                2 years       Debt consoli

In [188]:
df['Risk_Score'].isnull().any()

False

In [189]:
df.to_csv('updatedapprov.csv', index=False)


In [5]:
sql1='select * from LENDINGAI_DB.LENDINGAI_SCH.REJECTEDLOANAPPLICATIONS;'

In [6]:
res1=conn.cursor().execute(sql1).fetchall()

In [7]:
df1=pd.read_sql(sql1,conn)
df1.info()

C:\Users\ashritha.kanukula\AppData\Local\Temp\ipykernel_11144\2033228438.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1=pd.read_sql(sql1,conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8947707 entries, 0 to 8947706
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Application Date      object 
 1   Loan Title            object 
 2   Debt-To-Income Ratio  object 
 3   Zip Code              object 
 4   State                 object 
 5   Employment Length     object 
 6   Quarter               object 
 7   Label                 object 
 8   Amount Requested      float64
 9   Risk_Score            float64
 10  Policy Code           float64
 11  Year                  int64  
dtypes: float64(3), int64(1), object(8)
memory usage: 819.2+ MB


In [8]:
print(df1)

        Application Date          Loan Title Debt-To-Income Ratio Zip Code  \
0             2018-01-01  Debt consolidation               38.85%    488xx   
1             2018-01-01       Car financing                   0%    773xx   
2             2018-01-01            Business                0.55%    330xx   
3             2018-01-01         Home buying                 8.4%    307xx   
4             2018-01-01  Debt consolidation               24.22%    212xx   
...                  ...                 ...                  ...      ...   
8947702       2017-05-28  Debt consolidation               14.67%    711xx   
8947703       2017-05-28            Business               11.72%    270xx   
8947704       2017-05-28              moving                6.24%    276xx   
8947705       2017-05-28  Debt consolidation                36.2%    334xx   
8947706       2017-05-28  debt_consolidation               15.29%    531xx   

        State Employment Length Quarter     Label  Amount Reque

In [10]:
df1['Amount Requested'].isnull().any()

False

In [11]:
df1['Debt-To-Income Ratio'].isnull().any()

False

In [12]:
df1['Loan Title'].isnull().any()

True

In [13]:
df1['Label'].isnull().any()

False

In [14]:
df1['Employment Length'].isnull().any()

False

In [15]:
def convert_to_numerical(value):
    return float(value.strip('%')) / 100

# Apply the conversion function to the DataFrame column
df1['Debt-To-Income Ratio'] = df1['Debt-To-Income Ratio'].apply(convert_to_numerical)

# Format numerical values with precision (.2f)
df1['Debt-To-Income Ratio'] = df1['Debt-To-Income Ratio'].apply(lambda x: f'{x:.2f}')

# Display the updated DataFrame
print(df1)


        Application Date          Loan Title Debt-To-Income Ratio Zip Code  \
0             2018-01-01  Debt consolidation                 0.39    488xx   
1             2018-01-01       Car financing                 0.00    773xx   
2             2018-01-01            Business                 0.01    330xx   
3             2018-01-01         Home buying                 0.08    307xx   
4             2018-01-01  Debt consolidation                 0.24    212xx   
...                  ...                 ...                  ...      ...   
8947702       2017-05-28  Debt consolidation                 0.15    711xx   
8947703       2017-05-28            Business                 0.12    270xx   
8947704       2017-05-28              moving                 0.06    276xx   
8947705       2017-05-28  Debt consolidation                 0.36    334xx   
8947706       2017-05-28  debt_consolidation                 0.15    531xx   

        State Employment Length Quarter     Label  Amount Reque

In [16]:
df1.rename(columns={'Label': 'Application_status'}, inplace=True)

In [17]:
print(df1)

        Application Date          Loan Title Debt-To-Income Ratio Zip Code  \
0             2018-01-01  Debt consolidation                 0.39    488xx   
1             2018-01-01       Car financing                 0.00    773xx   
2             2018-01-01            Business                 0.01    330xx   
3             2018-01-01         Home buying                 0.08    307xx   
4             2018-01-01  Debt consolidation                 0.24    212xx   
...                  ...                 ...                  ...      ...   
8947702       2017-05-28  Debt consolidation                 0.15    711xx   
8947703       2017-05-28            Business                 0.12    270xx   
8947704       2017-05-28              moving                 0.06    276xx   
8947705       2017-05-28  Debt consolidation                 0.36    334xx   
8947706       2017-05-28  debt_consolidation                 0.15    531xx   

        State Employment Length Quarter Application_status  Amo

In [18]:
title_mapping = {
    'medical': 'medical expenses'
}

# Replace similar loan titles using the mapping
df1['Loan Title'] = df1['Loan Title'].replace(title_mapping)

# Display the updated DataFrame
print(df1)

        Application Date          Loan Title Debt-To-Income Ratio Zip Code  \
0             2018-01-01  Debt consolidation                 0.39    488xx   
1             2018-01-01       Car financing                 0.00    773xx   
2             2018-01-01            Business                 0.01    330xx   
3             2018-01-01         Home buying                 0.08    307xx   
4             2018-01-01  Debt consolidation                 0.24    212xx   
...                  ...                 ...                  ...      ...   
8947702       2017-05-28  Debt consolidation                 0.15    711xx   
8947703       2017-05-28            Business                 0.12    270xx   
8947704       2017-05-28              moving                 0.06    276xx   
8947705       2017-05-28  Debt consolidation                 0.36    334xx   
8947706       2017-05-28  debt_consolidation                 0.15    531xx   

        State Employment Length Quarter Application_status  Amo

In [19]:
column_names = df1.columns
print(column_names)

Index(['Application Date', 'Loan Title', 'Debt-To-Income Ratio', 'Zip Code',
       'State', 'Employment Length', 'Quarter', 'Application_status',
       'Amount Requested', 'Risk_Score', 'Policy Code', 'Year'],
      dtype='object')


In [22]:
column_name = 'Employment Length' 
value_to_find = 'n/a'
# Print rows with the specified value in the specified column 
rows_with_value = df1.loc[df1[column_name] == value_to_find] 

In [23]:
print(rows_with_value)

        Application Date          Loan Title Debt-To-Income Ratio Zip Code  \
258           2018-01-01               Other                 0.07    712xx   
435           2018-01-01  Debt consolidation                 0.06    437xx   
528           2018-01-01         Home buying                 0.22    370xx   
673           2018-01-01               Other                 0.25    532xx   
757           2018-01-01               Other                 0.23    128xx   
...                  ...                 ...                  ...      ...   
8947561       2017-05-28    home_improvement                 0.35    630xx   
8947591       2017-05-28         credit_card                 0.43    285xx   
8947608       2017-05-28               other                 0.00    212xx   
8947639       2017-05-28         credit_card                 0.24    786xx   
8947662       2017-05-28               other                 0.00    612xx   

        State Employment Length Quarter Application_status  Amo

In [24]:
column_name = 'Loan Title' 
value_to_find = 'other'
# Print rows with the specified value in the specified column 
rows_with_value = df1.loc[df1[column_name] == value_to_find]

In [25]:
print(rows_with_value)

        Application Date Loan Title Debt-To-Income Ratio Zip Code State  \
1131660       2018-02-24      other                 0.35    924xx    CA   
1869849       2017-09-16      other                 0.18    317xx    GA   
1869855       2017-09-16      other                 0.03    470xx    IN   
1869872       2017-09-16      other                 0.42    925xx    CA   
1869876       2017-09-16      other                 0.04    852xx    AZ   
...                  ...        ...                  ...      ...   ...   
8947662       2017-05-28      other                 0.00    612xx    IL   
8947669       2017-05-28      other                 0.00    337xx    FL   
8947671       2017-05-28      other                 0.14    480xx    MI   
8947691       2017-05-28      other                 0.24    027xx    MA   
8947694       2017-05-28      other                 0.08    432xx    OH   

        Employment Length Quarter Application_status  Amount Requested  \
1131660          < 1 year

In [26]:
column_name = 'Loan Title' 
value_to_find = 'Other'
# Print rows with the specified value in the specified column 
rows_with_value = df1.loc[df1[column_name] == value_to_find]

In [27]:
print(rows_with_value)

        Application Date Loan Title Debt-To-Income Ratio Zip Code State  \
6             2018-01-01      Other                 0.13    462xx    IN   
7             2018-01-01      Other                 0.02    984xx    WA   
10            2018-01-01      Other                 0.28    891xx    NV   
11            2018-01-01      Other                 0.47    310xx    GA   
19            2018-01-01      Other                 0.10    405xx    KY   
...                  ...        ...                  ...      ...   ...   
8947549       2017-05-28      Other                 1.00    745xx    OK   
8947555       2017-05-28      Other                 0.00    331xx    FL   
8947605       2017-05-28      Other                 1.00    300xx    GA   
8947653       2017-05-28      Other                 0.18    221xx    VA   
8947678       2017-05-28      Other                 0.25    240xx    VA   

        Employment Length Quarter Application_status  Amount Requested  \
6                < 1 year

In [28]:
column_name = 'Risk_Score'
null_count = df1[column_name].isnull().sum()
print(f"Number of null values in '{column_name}': {null_count}")

Number of null values in 'Risk_Score': 4717663


In [31]:
wanted_columns = ['Debt-To-Income Ratio', 'Employment Length','Loan Title','Application_status','Amount Requested','Risk_Score'] 
condition_column = 'Employment Length' 
condition_value = 'n/a'
# Extract specific columns and specific rows with the condition 
df1= df1.loc[df1[condition_column] != condition_value, wanted_columns] 
df1= df1.loc[(df1['Loan Title'].notnull())&(df1['Risk_Score'].notnull())&(df1[condition_column] != condition_value)&(df1['Loan Title'] != 'other')&(df1['Loan Title'] != 'Other'), wanted_columns]

In [32]:
print(df1)

        Debt-To-Income Ratio Employment Length          Loan Title  \
0                       0.39          < 1 year  Debt consolidation   
1                       0.00          < 1 year       Car financing   
2                       0.01          < 1 year            Business   
3                       0.08          < 1 year         Home buying   
4                       0.24          < 1 year  Debt consolidation   
...                      ...               ...                 ...   
8947699                 0.06           4 years  debt_consolidation   
8947701                 0.09          < 1 year  Debt consolidation   
8947702                 0.15          < 1 year  Debt consolidation   
8947703                 0.12          < 1 year            Business   
8947705                 0.36          < 1 year  Debt consolidation   

        Application_status  Amount Requested  Risk_Score  
0                 Rejected            6000.0       668.0  
1                 Rejected            500

In [33]:
df1['Debt-To-Income Ratio'].isnull().any()

False

In [34]:
df1['Risk_Score'].isnull().any()

False

In [35]:
df1.to_csv('rejected_before_stratified.csv', index=False)

In [36]:
df2= pd.read_csv('rejected_before_stratified.csv')

In [37]:
df2.head()

,Debt-To-Income Ratio,Employment Length,Loan Title,Application_status,Amount Requested,Risk_Score
0,0.39,< 1 year,Debt consolidation,Rejected,6000.0,668.0
1,0.00,< 1 year,Car financing,Rejected,5000.0,514.0
2,0.01,< 1 year,Business,Rejected,10000.0,603.0
3,0.08,< 1 year,Home buying,Rejected,10000.0,666.0
4,0.24,< 1 year,Debt consolidation,Rejected,15000.0,629.0


In [38]:
#Apply stratified sampling to overcome class imbalance
df2=df2.groupby('Application_status', group_keys=False).apply(lambda x: x.sample(frac=0.2))

In [39]:
print(df2) 

         Debt-To-Income Ratio Employment Length               Loan Title  \
1571907                  0.12          < 1 year  Credit card refinancing   
1287903                  0.06          < 1 year       Debt consolidation   
2763860                  0.31          < 1 year       Debt consolidation   
2455085                  0.01          < 1 year         Home improvement   
3141142                  0.06          < 1 year                 Business   
...                       ...               ...                      ...   
2510489                  0.15          < 1 year         Home improvement   
1375557                  0.04          < 1 year    Moving and relocation   
3273084                  0.19          < 1 year            Car financing   
1788300                  0.38          < 1 year  Credit card refinancing   
707570                   0.10         10+ years  Credit card refinancing   

        Application_status  Amount Requested  Risk_Score  
1571907           Rejected  

In [40]:
df2.to_csv('updatedreject.csv', index=False)

In [42]:
import pandas as pd

 

# Load the first dataset
data1 = pd.read_csv('updatedapprov.csv')

 

# Load the second dataset
data2 = pd.read_csv('updatedreject.csv')

 

# Concatenate the two datasets vertically
merge_data = pd.concat([data1, data2], ignore_index=True)

 

# Save the combined dataset to a new CSV file
merge_data.to_csv('final_dataset.csv', index=False)

In [43]:
print(merge_data)

         Amount Requested Application_status  Debt-To-Income Ratio  \
0                  6700.0           Approved                 20.00   
1                 18000.0           Approved                 25.09   
2                 12000.0           Approved                  4.90   
3                 10500.0           Approved                 35.57   
4                  8000.0           Approved                 18.39   
...                   ...                ...                   ...   
1166142           40000.0           Rejected                  0.15   
1166143            1000.0           Rejected                  0.04   
1166144           40000.0           Rejected                  0.19   
1166145           10000.0           Rejected                  0.38   
1166146           10000.0           Rejected                  0.10   

        Employment Length               Loan Title  Risk_Score  
0                 8 years       Debt consolidation       780.0  
1                 2 years    

In [44]:
#logistic regression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [45]:
data = pd.read_csv('final_dataset.csv')

In [46]:
X = data.drop('Application_status', axis=1)  # Replace 'target_column' with the actual target column name
y = data['Application_status']

In [47]:
model = LogisticRegression()

In [48]:
label_encoder = LabelEncoder()
X['Employment Length'] = label_encoder.fit_transform(X['Employment Length'])
X['Loan Title'] = label_encoder.fit_transform(X['Loan Title'])

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# Train the model on the training data
model.fit(X_train, y_train)

LogisticRegression()

In [51]:
# Make predictions on the test data
y_pred = model.predict(X_test)

In [52]:
#To find Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.96


In [53]:
# Display classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

    Approved       0.95      0.94      0.94     94515
    Rejected       0.96      0.96      0.96    138715

    accuracy                           0.96    233230
   macro avg       0.95      0.95      0.95    233230
weighted avg       0.96      0.96      0.96    233230

